In [1]:
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
import folium

In [2]:
from isochrone import (
    down_load_and_save_graph, get_origin_node, iso_polygon_from_nodes,
    get_base_map, add_poly
)

In [3]:
# lat, lon = 36.44930529404897, 140.60208035544244
# import_file_name = "jparc_10km"
# output_file_name = "jparc_10km"

# down_load_and_save_graph([lat, lon], dist=10_000, filename=f'{import_file_name}.graphml')

In [4]:
# lat, lon = 36.44930529404897, 140.60208035544244 # ニュートリノモニター棟
lat, lon = 36.46346973531239, 140.5966036136302 # J-PARCドミトリー
import_file_name = "jparc_10km"
output_file_name = "jparc_10km_walk"

G = ox.load_graphml(f"{import_file_name}.graphml")
print(f"グラフを読み込みました。ノード数: {len(G)}")

グラフを読み込みました。ノード数: 25105


In [5]:
origin, G_proj = get_origin_node(G, lat, lon)

In [6]:
df = pd.read_csv("jparc_food.csv")
df.head()

,lat,lon,name
0,36.460120,140.553315,パパディッシュ
1,36.476489,140.565794,○祐 まるすけ
2,36.468275,140.564828,日本料理 松臣
3,36.460892,140.594744,お食事処おるりば
4,36.492741,140.615327,海鮮茶屋 濱膳


In [ ]:
# --- 2) Dijkstra（距離: m） ---
speed_m_per_min = 60
thresh_5m  = 5  * speed_m_per_min
thresh_10m = 10 * speed_m_per_min
thresh_15m = 15 * speed_m_per_min
thresh_20m = 20 * speed_m_per_min
lengths = nx.single_source_dijkstra_path_length(
    G_proj, source=origin, cutoff=thresh_20m, weight="length"
)

# 到達ノード集合
S5  = {n for n, d in lengths.items() if d <= thresh_5m}
S10 = {n for n, d in lengths.items() if d <= thresh_10m}
S15 = {n for n, d in lengths.items() if d <= thresh_15m}
S20 = {n for n, d in lengths.items() if d <= thresh_20m}

# --- 3) エッジ由来でポリゴン化 ---
# 細め設定
poly_5m  = iso_polygon_from_nodes(G_proj, S5, buffer_m=6,  simplify_m=3)
poly_10m = iso_polygon_from_nodes(G_proj, S10, buffer_m=8, simplify_m=3)
poly_15m = iso_polygon_from_nodes(G_proj, S15, buffer_m=10, simplify_m=4)
poly_20m = iso_polygon_from_nodes(G_proj, S20, buffer_m=12, simplify_m=4)

# --- 4) Folium 可視化 ---
# レイヤ順：15 → 10 → 5（下から順に）
m = get_base_map([lat, lon], zoom_start=16)
add_poly(m, poly_20m, "徒歩20分圏（~1,200 m）", "#a9a9a9", fill_opacity=0.25)
add_poly(m, poly_15m, "徒歩15分圏（~8,490 m）",  "#4169e1", fill_opacity=0.25)
add_poly(m, poly_10m, "徒歩10分圏（~5,660 m）",  "#32cd32", fill_opacity=0.25)
add_poly(m, poly_5m,  "徒歩5分圏（~2,830 m）",   "#ff7f50", fill_opacity=0.25)

# 起点マーカー
folium.Marker([lat, lon], icon=folium.Icon(color="red"), tooltip="起点（J-PARCドミトリー）").add_to(m)
folium.LayerControl().add_to(m)

for _lat, _lon, name in zip(df["lat"], df["lon"], df["name"]):
    folium.Marker(
        [ _lat,  _lon],
        icon=folium.Icon(color="cadetblue", icon="cutlery", prefix="fa"),
        tooltip=name,
    ).add_to(m)

In [8]:
m.save(f"{output_file_name}.html")
print(f"{output_file_name}.html を出力しました。")

jparc_10km_walk.html を出力しました。
